In [2]:
from glob import glob
from astroquery.mast import Mast, Observations
import os
import shutil
import numpy as np
import json
import requests
import asdf
from astropy import log
from astropy.io import ascii, fits
from astropy.wcs import WCS
from astropy.coordinates import SkyCoord
from astropy.utils.data import download_file
from astropy.visualization import ImageNormalize, ManualInterval, LogStretch, LinearStretch
import matplotlib.pyplot as plt
import matplotlib as mpl
from astropy.table import Table
import astropy.units as u
import stdatamodels


# do this before importing webb
os.environ["CRDS_PATH"] = "/orange/adamginsburg/jwst/brick/crds/"
os.environ["CRDS_SERVER_URL"] = "https://jwst-crds.stsci.edu"

from jwst.pipeline import calwebb_image3
from jwst.pipeline import Detector1Pipeline, Image2Pipeline

# Individual steps that make up calwebb_image3
from jwst.tweakreg import TweakRegStep
from jwst.tweakreg.utils import adjust_wcs
from jwst.skymatch import SkyMatchStep
from jwst.outlier_detection import OutlierDetectionStep
from jwst.resample import ResampleStep
from jwst.source_catalog import SourceCatalogStep
from jwst import datamodels
from jwst.associations import asn_from_list
from jwst.associations.lib.rules_level3_base import DMS_Level3_Base
from jwst.datamodels import DataModel
from jwst.datamodels import ImageModel

import jwst.datamodels

In [3]:
filtername = "F410M"

In [4]:
files = glob(f"/orange/adamginsburg/jwst/cloudc/{filtername}/pipeline/*nrcb*long_destreak.fits")
files

['/orange/adamginsburg/jwst/cloudc/F410M/pipeline/jw02221002002_12201_00003_nrcblong_destreak.fits',
 '/orange/adamginsburg/jwst/cloudc/F410M/pipeline/jw02221002001_12201_00001_nrcblong_destreak.fits',
 '/orange/adamginsburg/jwst/cloudc/F410M/pipeline/jw02221002001_12201_00002_nrcblong_destreak.fits',
 '/orange/adamginsburg/jwst/cloudc/F410M/pipeline/jw02221002001_06201_00003_nrcblong_destreak.fits',
 '/orange/adamginsburg/jwst/cloudc/F410M/pipeline/jw02221002002_12201_00001_nrcblong_destreak.fits',
 '/orange/adamginsburg/jwst/cloudc/F410M/pipeline/jw02221002001_12201_00003_nrcblong_destreak.fits',
 '/orange/adamginsburg/jwst/cloudc/F410M/pipeline/jw02221002001_12201_00004_nrcblong_destreak.fits',
 '/orange/adamginsburg/jwst/cloudc/F410M/pipeline/jw02221002001_06201_00001_nrcblong_destreak.fits',
 '/orange/adamginsburg/jwst/cloudc/F410M/pipeline/jw02221002002_12201_00002_nrcblong_destreak.fits',
 '/orange/adamginsburg/jwst/cloudc/F410M/pipeline/jw02221002002_06201_00002_nrcblong_destre

In [ ]:
filename = files[0]
align_image = filename.replace("_destreak.fits", "_align.fits")
shutil.copy(filename, align_image)
offsets_tbl = Table.read('/orange/adamginsburg/jwst/cloudc/offsets/Offsets_JWST_Cloud_C.csv')
row = offsets_tbl[filename.split('/')[-1] == offsets_tbl['Filename_1']]
print('Running manual align.')
try:
    xshift = float(row['xshift (arcsec)'])*u.arcsec
    yshift = float(row['yshift (arcsec)'])*u.arcsec
except:
    log.info('Something went wrong with manual align, running default values.')
    visit = filename.split('_')[0][-3:]
    if visit == '001':
        xshift = 8*u.arcsec
        yshift = -0.3*u.arcsec
    elif visit == '002':
        xshift = 3.9*u.arcsec
        yshift = 1.5*u.arcsec
    else:
        xshift = 0*u.arcsec
        yshift = 0*u.arcsec
align_fits = ImageModel(align_image)
ww = adjust_wcs(align_fits.meta.wcs, delta_ra = yshift, delta_dec = xshift)
align_fits.meta.wcs = ww
align_fits.save(align_image)
align_fits = fits.open(align_image)
align_fits[1].header.update(ww.to_fits()[0])
align_fits.writeto(align_image, overwrite=True)
filename = align_image

In [ ]:
'''
for f in files:
    align_image = member['expname'].replace("_destreak.fits", "_align.fits")
    shutil.copy(member['expname'], align_image)
    offsets_tbl = Table.read('/orange/adamginsburg/jwst/cloudc/offsets/Offsets_JWST_Cloud_C.csv')
    row = offsets_tbl[member['expname'].split('/')[-1] == offsets_tbl['Filename_1']]
    print('Running manual align.')
    try:
        xshift = float(row['xshift (arcsec)'])*u.arcsec
        yshift = float(row['yshift (arcsec)'])*u.arcsec
    except:
        log.info('Something went wrong with manual align, running default values.')
        visit = member['expname'].split('_')[0][-3:]
        if visit == '001':
            xshift = 8*u.arcsec
            yshift = -0.3*u.arcsec
        elif visit == '002':
            xshift = 3.9*u.arcsec
            yshift = 1*u.arcsec
        else:
            xshift = 0*u.arcsec
            yshift = 0*u.arcsec
    align_fits = ImageModel(align_image)
    ww = adjust_wcs(align_fits.meta.wcs, delta_ra = yshift, delta_dec = xshift)
    align_fits.meta.wcs = ww
    align_fits.save(align_image)
    align_fits = fits.open(align_image)
    align_fits[1].header.update(ww.to_fits()[0])
    align_fits.writeto(align_image, overwrite=True)
'''

In [ ]:
for f in files:
    filename = f
    align_image = filename.replace("_destreak.fits", "_align.fits")
    shutil.copy(filename, align_image)
    offsets_tbl = Table.read('/orange/adamginsburg/jwst/cloudc/offsets/Offsets_JWST_Cloud_C.csv')
    row = offsets_tbl[filename.split('/')[-1] == offsets_tbl['Filename_1']]
    print('Running manual align.')
    try:
        xshift = float(row['xshift (arcsec)'])*u.arcsec
        yshift = float(row['yshift (arcsec)'])*u.arcsec
    except:
        log.info('Something went wrong with manual align, running default values.')
        visit = filename.split('_')[0][-3:]
        if visit == '001':
            xshift = 8*u.arcsec
            yshift = -0.3*u.arcsec
        elif visit == '002':
            xshift = 3.9*u.arcsec
            yshift = 1*u.arcsec
        else:
            xshift = 0*u.arcsec
            yshift = 0*u.arcsec
    align_fits = ImageModel(align_image)
    ww = adjust_wcs(align_fits.meta.wcs, delta_ra = yshift+0.6*u.arcsec, delta_dec = xshift)
    align_fits.meta.wcs = ww
    align_fits.save(align_image)
    align_fits = fits.open(align_image)
    align_fits[1].header.update(ww.to_fits()[0])
    align_fits.writeto(align_image, overwrite=True)
    #filename = align_image

In [ ]:
print('done')

In [ ]:
glob(f"/orange/adamginsburg/jwst/cloudc/{filtername}/pipeline/*nrc*long_align.fits")

In [5]:
models = jwst.datamodels.open(glob(f"/orange/adamginsburg/jwst/cloudc/{filtername}/pipeline/*nrc*long_align.fits"))

In [6]:
#abs_refcat = '/orange/adamginsburg/jwst/cloudc/catalogs/crowdsource_based_nircam-f405n_reference_astrometric_catalog.ecsv'
abs_refcat = '/orange/adamginsburg/jwst/cloudc/F405N/pipeline/jw02221-o002_t001_nircam_clear-F405N-merged_vvvcat.fits'

In [ ]:
ls /orange/adamginsburg/jwst/cloudc/F405N/pipeline/*vvvcat*

In [9]:
tweakreg = TweakRegStep()
tweakreg.fitgeometry = 'general'
tweakreg.brightest = 500
tweakreg.snr_threshold = 30
tweakreg.abs_refcat = abs_refcat
tweakreg.save_catalogs = True
tweakreg.catalog_format = 'ecsv'
#tweakreg.kernel_fwhm = fwhm_pix
tweakreg.nclip = 5
tweakreg.sharplo = 0.3
tweakreg.sharphi = 0.9
tweakreg.roundlo = -0.25
tweakreg.roundhi = 0.25
tweakreg.separation = 0.5
tweakreg.output_dir = f"/orange/adamginsburg/jwst/cloudc/{filtername}/pipeline/"#output_dir
tweakreg.save_results = True
tweakreg.skip = False
tweakreg.searchrad = 0.5

tweak = tweakreg.run(models)


2023-12-06 15:49:30,198 - stpipe.TweakRegStep - INFO - TweakRegStep instance created.
2023-12-06 15:49:30,738 - stpipe.TweakRegStep - INFO - Step TweakRegStep running with args (<ModelContainer>,).
2023-12-06 15:49:30,746 - stpipe.TweakRegStep - INFO - Step TweakRegStep parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': '/orange/adamginsburg/jwst/cloudc/F410M/pipeline/', 'output_ext': '.fits', 'output_use_model': True, 'output_use_index': True, 'save_results': True, 'skip': False, 'suffix': None, 'search_output_file': True, 'input_dir': '', 'save_catalogs': True, 'use_custom_catalogs': False, 'catalog_format': 'ecsv', 'catfile': '', 'kernel_fwhm': 2.5, 'snr_threshold': 30, 'sharplo': 0.3, 'sharphi': 0.9, 'roundlo': -0.25, 'roundhi': 0.25, 'brightest': 500, 'peakmax': None, 'bkg_boxsize': 400, 'enforce_user_order': False, 'expand_refcat': False, 'minobj': 15, 'searchrad': 0.5, 'use2dhist': True, 'separation': 0.5, 'tolerance': 0.7, 'xoffset': 0.0, 'yo

KeyboardInterrupt: 

In [ ]:
print('done')